This file is the preprocessing part. We process the data into a useable one to put into model. The approach is mostly based on milestone 1.2.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
raw_claims = pd.read_csv('Data/Claims.csv')
raw_claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [3]:
raw_claims.size

37365860

In [4]:
def rmsle_metric(y_actual, y_predicted):
    y_actual_log = np.log(y_actual + 1)
    y_predicted_log = np.log(y_predicted + 1)
    rms = sqrt(mean_squared_error(y_actual_log, y_predicted_log))
    return rms

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
    
def under_over_sample(X, y):
    smote_enn = SMOTEENN(random_state=0, sampling_strategy='all')
    X_resampled, y_resampled = smote_enn.fit_resample(X, y)
    return X_resampled, y_resampled

def over_sample(X, y, method='smote'):
    if method == 'smote':
        X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X, y)
    elif method == 'adasyn':
        X_resampled, y_resampled = ADASYN(random_state=0).fit_resample(X, y)
    return X_resampled, y_resampled
    
def export_model(model, file_name):
    pickle.dump(model, open(file_name, 'wb'))
    
def import_model(file_name):
    loaded_model = pickle.load(open(file_name, 'rb'))
    return loaded_model
    
dsfs_switcher = {
    '0- 1 month': 1,
    '1- 2 months': 2,
    '2- 3 months': 3,
    '3- 4 months': 4,
    '4- 5 months': 5,
    '5- 6 months': 6,
    '6- 7 months': 7,
    '7- 8 months': 8,
    '8- 9 months': 9,
    '9-10 months': 10,
    '10-11 months': 11,
    '11-12 months': 12,
    np.nan: np.nan
}

def cast_dsfs_to_int(string):
    return dsfs_switcher.get(string)

charlson_switcher = {
    '0': 0,
    '1-2': 2,
    '3-4': 4,
    '5+': 6
}

def cast_charlson_to_int(string):
    return charlson_switcher.get(string)

length_of_stay_switcher = {
    '1 day': 1,
    '2 days': 2,
    '3 days': 3,
    '4 days': 4,
    '5 days': 5,
    '6 days': 6,
    '1- 2 weeks': 11,
    '2- 4 weeks': 21,
    '4- 8 weeks': 42,
    '26+ weeks': 180,
    np.nan: np.nan
}

def cast_length_of_stay_to_int(string):
    return length_of_stay_switcher.get(string)

summary_table_column_names = ['MemberID', 'Year', 'no_Claims']
primary_condition_group_values = ['MSC2a3', 'METAB3', 'ARTHSPIN', 'NEUMENT', 'RESPR4', 'MISCHRT', 
                                  'SKNAUT', 'GIBLEED', 'INFEC4', 'TRAUMA', 'HEART2', 'RENAL3', 
                                  'ROAMI', 'MISCL5', 'ODaBNCA', 'UTI', 'COPD', 'FXDISLC', 'PNEUM',
                                  'GIOBSENT', 'GYNEC1', 'AMI', 'HIPFX', 'CANCRB', 'SEIZURE', 'CHF', 'PRGNCY',
                                   'METAB1', 'HEART4', 'HEMTOL', 'GYNECA', 'APPCHOL',
                                   'MISCL1', 'RENAL2', 'STROKE', 'PERVALV', 'CANCRA', 'CATAST',
                                    'FLaELEC', 'SEPSIS', 'LIVERDZ', 'PNCRDZ', 'RENAL1', 'CANCRM',
                                   'PERINTL']
procedure_group_values = ['MED', 'EM', 'SCS', 'RAD', 'ANES', 'SDS', 'PL', 'SIS', 'SMS',
       'SGS', 'SEOA', 'SNS', 'SAS', 'SRS', 'SUS', 'SO', 'SMCD']
specialty_values = ['Surgery', 'Internal', 'Laboratory', 'Pediatrics',
       'Rehabilitation', 'Diagnostic Imaging', 'Anesthesiology',
       'Emergency', 'General Practice', 'Other',
       'Obstetrics and Gynecology', 'Pathology']
place_service_values = ['Office', 'Independent Lab', 'Outpatient Hospital',
       'Inpatient Hospital', 'Urgent Care', 'Ambulance', 'Home',
       'Other']
provider_id_values = raw_claims["ProviderID"].value_counts().index.tolist()[:46]
vendor_list = raw_claims["Vendor"].value_counts().index.tolist()[:42]
categorical_column_name = list()

In [5]:
raw_claims['PayDelay'] = raw_claims['PayDelay'].replace('162+', '163').astype(np.int64)
raw_claims["CharlsonIndex"] = raw_claims["CharlsonIndex"].map(cast_charlson_to_int)
raw_claims["LengthOfStay"] = raw_claims["LengthOfStay"].map(cast_length_of_stay_to_int)
raw_claims["DSFS"] = raw_claims["DSFS"].map(cast_dsfs_to_int)
raw_claims["los_tot_unknown"] = (raw_claims["LengthOfStay"].isnull()) & (raw_claims["SupLOS"] == 0)
raw_claims["los_tot_suppressed"] = (raw_claims["LengthOfStay"].isnull()) & (raw_claims["SupLOS"] == 1)
raw_claims["los_tot_known"] = (~raw_claims["LengthOfStay"].isnull())
categorical_column_name.extend(["los_tot_unknown", "los_tot_suppressed", "los_tot_known"])
raw_claims["pcg0"] =  raw_claims["PrimaryConditionGroup"].isnull()
categorical_column_name.append("pcg0")
for indx, val in enumerate(primary_condition_group_values):
    name = "pcg" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["PrimaryConditionGroup"] == val
raw_claims["sp0"] = raw_claims["Specialty"].isnull()
categorical_column_name.append("sp0")
for indx, val in enumerate(specialty_values):
    name = "sp" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["Specialty"] == val
raw_claims["pg0"] = raw_claims["ProcedureGroup"].isnull()
categorical_column_name.append("pg0")
for indx, val in enumerate(procedure_group_values):
    name = "pg" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["ProcedureGroup"] == val
raw_claims["ps0"] = raw_claims["PlaceSvc"].isnull()
categorical_column_name.append("ps0")
for indx, val in enumerate(place_service_values):
    name = "ps" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["PlaceSvc"] == val 
raw_claims["pid46"] = ~raw_claims["ProviderID"].isin(provider_id_values)
categorical_column_name.append("pid46")
raw_claims["pid0"] =  raw_claims["ProviderID"].isnull()
categorical_column_name.append("pid0")
for indx, val in enumerate(provider_id_values):
    name = "pid" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["ProviderID"] == val
raw_claims["vid46"] = ~raw_claims["Vendor"].isin(provider_id_values)
categorical_column_name.append("vid42")
raw_claims["vid0"] =  raw_claims["Vendor"].isnull()
categorical_column_name.append("vid0")
for indx, val in enumerate(provider_id_values):
    name = "vid" + str(indx+1)
    categorical_column_name.append(name)
    raw_claims[name] = raw_claims["Vendor"] == val

In [6]:
raw_claims["LengthOfStay"].value_counts(dropna=False)

NaN       2597392
 1.0        56696
 2.0         6485
 3.0         3246
 4.0         1473
 11.0        1143
 21.0         961
 42.0         903
 5.0          510
 6.0          179
 180.0          2
Name: LengthOfStay, dtype: int64

In [7]:
raw_claims.dtypes.value_counts()

bool       183
object       5
float64      5
int64        4
dtype: int64

In [8]:
# Missing values statistics
missing_values = missing_values_table(raw_claims)
missing_values

Your selected dataframe has 197 columns.
There are 9 columns that have missing values.


,Missing Values,% of Total Values
LengthOfStay,2597392,97.3
DSFS,52770,2.0
Vendor,24856,0.9
ProviderID,16264,0.6
PrimaryConditionGroup,11410,0.4
Specialty,8405,0.3
PlaceSvc,7632,0.3
PCP,7492,0.3
ProcedureGroup,3675,0.1


In [9]:
#I choose paydelay for counting # Claims because it doesn't have missing value
summary_table_count = raw_claims.groupby( [ "MemberID", "Year"] ).count()["PayDelay"]
summary_table_count.rename("no_Claims", inplace=True)
summary_table_property_count = raw_claims.groupby( [ "MemberID", "Year"])["ProviderID", "Vendor", "PCP", "PlaceSvc", "Specialty", "PrimaryConditionGroup", "ProcedureGroup"].nunique()
summary_table_property_count.rename(columns = {
    "ProviderID": "no_Providers", 
    "Vendor":"no_Vendors",
    "PCP":"no_PCPs",
    "PlaceSvc":"no_PlaceSvcs",
    "Specialty":"no_Specialities",
    "PrimaryConditionGroup":"no_PrimaryConditionGroups",
    "ProcedureGroup":"no_ProcedureGroups",
}, inplace = True) 
summary_table_property_max = raw_claims.groupby( [ "MemberID", "Year"])["PayDelay", "LengthOfStay", "DSFS", "CharlsonIndex"].max()
summary_table_property_max.rename(columns = {
    "PayDelay": "PayDelay_max", 
    "LengthOfStay":"LOS_max",
    "DSFS":"dsfs_max",
    "CharlsonIndex":"CharlsonIndexI_max",
}, inplace = True) 
summary_table_property_min = raw_claims.groupby( [ "MemberID", "Year"])["PayDelay", "LengthOfStay", "DSFS", "CharlsonIndex"].min()
summary_table_property_min.rename(columns = {
    "PayDelay": "PayDelay_min", 
    "LengthOfStay":"LOS_min",
    "DSFS":"dsfs_min",
    "CharlsonIndex":"CharlsonIndexI_min",
}, inplace = True) 
summary_table_property_ave = raw_claims.groupby( [ "MemberID", "Year"])["PayDelay", "LengthOfStay", "DSFS", "CharlsonIndex"].mean()
summary_table_property_ave.rename(columns = {
    "PayDelay": "PayDelay_ave", 
    "LengthOfStay":"LOS_ave",
    "DSFS":"dsfs_ave",
    "CharlsonIndex":"CharlsonIndexI_ave",
}, inplace = True) 
summary_table_property_std = raw_claims.groupby( [ "MemberID", "Year"])["PayDelay", "LengthOfStay", "DSFS", "CharlsonIndex"].std()
summary_table_property_std.rename(columns = {
    "PayDelay": "PayDelay_stdev", 
    "LengthOfStay":"LOS_stdev",
    "DSFS":"dsfs_stdev",
    "CharlsonIndex":"CharlsonIndexI_stdev",
}, inplace = True)
summary_table_dsfs_range = raw_claims.groupby( [ "MemberID", "Year"])["DSFS"].agg(['min', 'max'])
summary_table_dsfs_range['dsfs_range'] = summary_table_dsfs_range['max'] - summary_table_dsfs_range['min']
summary_table_dsfs_range.drop(['max', 'min'], axis=1, inplace=True)
summary_table_charlson_index_range = raw_claims.groupby( [ "MemberID", "Year"])["CharlsonIndex"].agg(['min', 'max'])
summary_table_charlson_index_range['CharlsonIndexI_range'] = summary_table_charlson_index_range['max'] - summary_table_charlson_index_range['min']
summary_table_charlson_index_range.drop(['max', 'min'], axis=1, inplace=True)
summary_table_property_sum = raw_claims.groupby( [ "MemberID", "Year"])[categorical_column_name].sum()


In [10]:
claims_per_member = pd.concat([summary_table_count, 
                 summary_table_property_count, 
                 summary_table_property_max, 
                 summary_table_property_min,
                 summary_table_property_ave, 
                 summary_table_property_std, 
                 summary_table_dsfs_range,
                 summary_table_charlson_index_range, 
                 summary_table_property_sum
                ], axis=1)
claims_per_member.sort_index()

no_Claims  no_Providers  no_Vendors  no_PCPs  no_PlaceSvcs  \
MemberID Year                                                               
4        Y2            1             1           1        1             1   
210      Y1            8             4           4        2             3   
         Y2            6             3           3        1             2   
         Y3            4             2           2        1             2   
3197     Y1            5             3           3        1             2   
         Y2            5             4           3        1             2   
         Y3           11             3           1        1             1   
3457     Y2            1             1           1        1             1   
3713     Y2           10             5           5        2             2   
3741     Y2           20             3           3        1             2   
3889     Y1           13             7           7        1             3   
4048     Y3           22            10           9        1             5   
4187     Y1            4             3           3        1             1   
5187     Y2            2             1           1        1             1   
         Y3           13             4           3        1             2   
8213     Y2            3             2           2        1             2   
9063     Y1            4             2           2        1             1   
10242    Y2           14             4           4        2             3   
11951    Y1            6             3           3        1             2   
         Y2           11             8           8        1             3   
14033    Y2            4             1           1        1             1   
         Y3            8             7           7        1             2   
14552    Y3            8             6           5        1             2   
14661    Y1            2             2           2        1             2   
         Y2           12             4           4        1             3   
         Y3            3             1           1        1             1   
14701    Y1            3             2           2        1             1   
14778    Y1            6             5           4        2             2   
         Y2            8             6           5        2             2   
         Y3            8             2           2        1             2   
...                  ...           ...         ...      ...           ...   
99977491 Y1            8             5           5        1             1   
         Y2            6             2           2        1             1   
         Y3           14            11           9        1             4   
99979211 Y1            2             1           1        1             1   
99979429 Y3            2             2           2        1             1   
99982956 Y2            5             3           3        1             3   
         Y3            2             1           1        1             1   
99985652 Y1            2             2           2        1             1   
99985795 Y2            3             2           2        1             1   
         Y3            2             1           1        1             1   
99985996 Y2            5             2           2        1             1   
         Y3           18             6           5        1             3   
99987030 Y2            4             3           2        1             1   
         Y3            8             3           3        1             2   
99987318 Y1            3             2           2        1             2   
99988469 Y1           12             6           3        1             4   
99992565 Y1            8             3           2        1             2   
         Y3           25             2           2        1             2   
99993334 Y3            2             1           1        1             1   
99994536 Y1

In [11]:
claims_per_member["LOS_max"].fillna(0, inplace=True)
claims_per_member["LOS_min"].fillna(0, inplace=True)
claims_per_member["LOS_ave"].fillna(0, inplace=True)
claims_per_member["LOS_stdev"].fillna(-1, inplace=True)
claims_per_member["dsfs_max"].fillna(0, inplace=True)
claims_per_member["dsfs_min"].fillna(0, inplace=True)
claims_per_member["dsfs_ave"].fillna(0, inplace=True)
claims_per_member["dsfs_stdev"].fillna(-1, inplace=True)
claims_per_member["dsfs_range"].fillna(-1, inplace=True)
claims_per_member["CharlsonIndexI_range"].fillna(-1, inplace=True)

In [12]:
missing_values = missing_values_table(claims_per_member)
missing_values

Your selected dataframe has 211 columns.
There are 11 columns that have missing values.


,Missing Values,% of Total Values
LOS_stdev,203046,93.0
LOS_max,174113,79.7
LOS_min,174113,79.7
LOS_ave,174113,79.7
dsfs_stdev,18627,8.5
PayDelay_stdev,18443,8.4
CharlsonIndexI_stdev,18443,8.4
dsfs_max,52,0.0
dsfs_min,52,0.0
dsfs_ave,52,0.0


In [13]:
claims_per_member.to_csv('Data/claims_per_member.csv', encoding='utf-8')

In [14]:
claims_per_member = pd.read_csv('Data/claims_per_member.csv')
claims_per_member.set_index(['MemberID', 'Year'], inplace=True)
claims_per_member.head()

no_Claims  no_Providers  no_Vendors  no_PCPs  no_PlaceSvcs  \
MemberID Year                                                               
4        Y2            1             1           1        1             1   
210      Y1            8             4           4        2             3   
         Y2            6             3           3        1             2   
         Y3            4             2           2        1             2   
3197     Y1            5             3           3        1             2   

               no_Specialities  no_PrimaryConditionGroups  no_ProcedureGroups  \
MemberID Year                                                                   
4        Y2                  1                          1                   1   
210      Y1                  3                          4                   5   
         Y2                  3                          2                   3   
         Y3                  2                          2                   2   
3197     Y1                  2                          2                   2   

               PayDelay_max  LOS_max  ...    vid37  vid38  vid39  vid40  \
MemberID Year                         ...                                 
4        Y2              43      NaN  ...      0.0    0.0    0.0    0.0   
210      Y1             163      2.0  ...      0.0    0.0    0.0    0.0   
         Y2             128      NaN  ...      0.0    0.0    0.0    0.0   
         Y3              44      NaN  ...      0.0    0.0    0.0    0.0   
3197     Y1             163      NaN  ...      0.0    0.0    0.0    0.0   

               vid41  vid42.1  vid43  vid44  vid45  vid46  
MemberID Year                                              
4        Y2      0.0      0.0    0.0    0.0    0.0    0.0  
210      Y1      0.0      0.0    0.0    0.0    0.0    0.0  
         Y2      0.0      0.0    0.0    0.0    0.0    0.0  
         Y3      0.0      0.0    0.0    0.0    0.0    0.0  
3197     Y1      0.0      0.0    0.0    0.0    0.0    0.0  

[5 rows x 211 columns]

In [15]:
Members = pd.read_csv('Data/Members.csv')
Members.head()

,MemberID,AgeAtFirstClaim,Sex
0,14723353,70-79,M
1,75706636,70-79,M
2,17320609,70-79,M
3,69690888,40-49,M
4,33004608,0-9,M


In [16]:
members_age_values = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
for indx, val in enumerate(members_age_values):
    name = "age_" + str(indx) + "5" 
    Members[name] = Members["AgeAtFirstClaim"] == val
Members["age_MISS"] = Members["AgeAtFirstClaim"].isnull()

In [17]:
Members["sexMALE"] = Members["Sex"] == 'M'
Members["sexFEMALE"] = Members["Sex"] == 'F'
Members["sexMISS"] = Members["Sex"].isnull()

In [18]:
Members.to_csv('Data/Members_processed.csv', encoding='utf-8')

In [19]:
LabCount = pd.read_csv('Data/LabCount.csv')
LabCount.head()

,MemberID,Year,DSFS,LabCount
0,69258001,Y3,2- 3 months,1
1,10143167,Y1,0- 1 month,2
2,1054357,Y1,0- 1 month,6
3,56583841,Y3,6- 7 months,4
4,70967047,Y2,0- 1 month,2


In [20]:
LabCount['LabCount'] = LabCount['LabCount'].replace('10+', '10').astype(np.int64)

In [21]:
LabCountSummary = LabCount.groupby( [ "MemberID", "Year"])["LabCount"].agg(['min', 'max', 'mean', 'count'])
LabCountSummary.rename(columns = {
    "max": "labCount_max", 
    "min": "labCount_min", 
    "mean": "labCount_ave", 
    "count": "labcount_months", 
}, inplace = True) 
LabCountSummary.head()

labCount_min  labCount_max  labCount_ave  labcount_months
MemberID Year                                                           
210      Y1               2             2           2.0                1
         Y2               1             1           1.0                1
         Y3               1             1           1.0                1
3197     Y2               2             2           2.0                1
3713     Y2               1             8           4.5                2

In [22]:
LabCountSummary.to_csv('Data/LabCountSummary.csv', encoding='utf-8') 

In [23]:
DaysInHospital_Y2 = pd.read_csv('Data/DaysInHospital_Y2.csv')
DaysInHospital_Y3 = pd.read_csv('Data/DaysInHospital_Y3.csv')
DaysInHospital_Y3.head()

,MemberID,ClaimsTruncated,DaysInHospital
0,90963501,0,0
1,85160905,0,0
2,45496648,0,0
3,45162557,0,0
4,68793937,0,0


In [24]:
DaysInHospital_Y2["Year"]='Y1'
DaysInHospital_Y3["Year"]='Y2'

In [25]:
DaysInHospital_Y2.set_index(['MemberID'], inplace=True)
DaysInHospital_Y3.set_index(['MemberID'], inplace=True)

In [26]:
Members_processed = pd.read_csv('Data/Members_processed.csv')
Members_processed.drop(['Unnamed: 0'], axis=1, inplace=True)
Members_processed.set_index(['MemberID'], inplace=True)
Members_processed.sort_index().head(10)

,AgeAtFirstClaim,Sex,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,age_85,age_MISS,sexMALE,sexFEMALE,sexMISS
MemberID,,,,,,,,,,,,,,,
4,0-9,M,True,False,False,False,False,False,False,False,False,False,True,False,False
210,30-39,NaN,False,False,False,True,False,False,False,False,False,False,False,False,True
3197,0-9,F,True,False,False,False,False,False,False,False,False,False,False,True,False
3457,0-9,M,True,False,False,False,False,False,False,False,False,False,True,False,False
3713,40-49,F,False,False,False,False,True,False,False,False,False,False,False,True,False
3741,70-79,F,False,False,False,False,False,False,False,True,False,False,False,True,False
3889,NaN,F,False,False,False,False,False,False,False,False,False,True,False,True,False
4048,50-59,M,False,False,False,False,False,True,False,False,False,False,True,False,False
4187,50-59,F,False,False,False,False,False,True,False,False,False,False,False,True,False


In [ ]:
DrugCount_full = pd.read_csv('Data/DrugCount.csv')
DrugCount_full['DrugCount'] = DrugCount_full['DrugCount'].replace('7+', '7').astype(np.int64)
DrugCount_full.drop(['DSFS'], axis=1, inplace=True)
DrugCount =DrugCount_full.groupby( [ "MemberID", "Year"])["DrugCount"].agg(['min', 'max', 'mean', 'count'])
DrugCount.rename(columns = {
    "max": "DrugCountMax", 
    "min": "DrugCountMin", 
    "mean": "DrugCountMean", 
    "count": "DrugCount", 
}, inplace = True)
DrugCount.to_csv('Data/df_DrugCount.csv')

In [49]:
drug_count = pd.read_csv('Data/df_DrugCount.csv')
drop_columns_drug_count = ['DSFSDrugCountMax',
 'DSFSDrugCountMean',
 'DSFSDrugCountMin',
 'DSFSDrugCountRange',
 'DrugCountRange']
for column in drop_columns_drug_count:
    if column in drug_count.columns:
        drug_count.drop([column], axis=1, inplace=True)
drug_count = drug_count.groupby(['MemberID', 'Year']).sum()
drug_count.sort_index().head(10)

DrugCount  DrugCountMax  DrugCountMin  DrugCountMean
MemberID Year                                                      
210      Y1            3             2             1       1.666667
         Y3            4             2             1       1.250000
3197     Y1            4             2             1       1.250000
         Y2            2             2             1       1.500000
         Y3            5             2             1       1.200000
3713     Y2            4             6             1       4.250000
3741     Y2           11             5             3       3.181818
3889     Y1           10             5             2       3.000000
4048     Y3            1             1             1       1.000000
4187     Y1           10             7             3       6.100000

In [28]:
LabCountSummary = pd.read_csv('Data/LabCountSummary.csv')
LabCountSummary = LabCountSummary.groupby(['MemberID', 'Year']).sum()
LabCountSummary.sort_index().head(10)

labCount_min  labCount_max  labCount_ave  labcount_months
MemberID Year                                                           
210      Y1               2             2           2.0                1
         Y2               1             1           1.0                1
         Y3               1             1           1.0                1
3197     Y2               2             2           2.0                1
3713     Y2               1             8           4.5                2
3741     Y2               1             5           2.2                5
3889     Y1              10            10          10.0                1
4048     Y3               6            10           8.0                2
5187     Y3               1             1           1.0                1
8213     Y2               4             4           4.0                1

In [29]:
temp_member_dih_train = pd.concat([DaysInHospital_Y2, 
                 Members_processed
                ], join='inner', axis=1)
temp_member_dih_test = pd.concat([DaysInHospital_Y3, 
                 Members_processed
                ], join='inner', axis=1)

In [30]:
temp_member_dih_train.reset_index(inplace=True)
temp_member_dih_train = temp_member_dih_train.groupby(['MemberID', 'Year']).sum()
temp_member_dih_test.reset_index(inplace=True)
temp_member_dih_test = temp_member_dih_test.groupby(['MemberID', 'Year']).sum()
temp_member_dih_test.head()

,,ClaimsTruncated,DaysInHospital,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,age_85,age_MISS,sexMALE,sexFEMALE,sexMISS
MemberID,Year,,,,,,,,,,,,,,,
4,Y2,0,0,True,False,False,False,False,False,False,False,False,False,True,False,False
210,Y2,0,0,False,False,False,True,False,False,False,False,False,False,False,False,True
3197,Y2,0,0,True,False,False,False,False,False,False,False,False,False,False,True,False
3457,Y2,0,0,True,False,False,False,False,False,False,False,False,False,True,False,False
3713,Y2,0,0,False,False,False,False,True,False,False,False,False,False,False,True,False


In [31]:
temp_member_claim_train = pd.concat([temp_member_dih_train, 
                        claims_per_member
                ], join='inner', axis=1)
temp_member_claim_test = pd.concat([temp_member_dih_test, 
                        claims_per_member
                ], join='inner', axis=1)
temp_member_claim_test.sort_index()

,,ClaimsTruncated,DaysInHospital,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,...,vid37,vid38,vid39,vid40,vid41,vid42.1,vid43,vid44,vid45,vid46
MemberID,Year,,,,,,,,,,,,,,,,,,,,,
4,Y2,0,0,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,Y2,0,0,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3197,Y2,0,0,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3457,Y2,0,0,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3713,Y2,0,0,False,False,False,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3741,Y2,0,0,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5187,Y2,0,0,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8213,Y2,0,0,False,True,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10242,Y2,0,0,False,False,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
temp_drug_claim_train = pd.merge(temp_member_claim_train, drug_count, how='left', on=['MemberID', 'Year'])
train_data = pd.merge(temp_drug_claim_train, LabCountSummary, how='left', on=['MemberID', 'Year'])
temp_drug_claim_test = pd.merge(temp_member_claim_test, drug_count, how='left', on=['MemberID', 'Year'])
test_data = pd.merge(temp_drug_claim_test, LabCountSummary, how='left', on=['MemberID', 'Year'])

In [33]:
train_data["labNull"] = train_data["labCount_max"] == 0
train_data["drugNull"] = train_data["DrugCountMax"] == 0
train_data = train_data.fillna(0)
test_data["labNull"] = test_data["labCount_max"] == 0
test_data["drugNull"] = test_data["DrugCountMax"] == 0
test_data = test_data.fillna(0)

In [34]:
missing_values = missing_values_table(train_data)
missing_values

Your selected dataframe has 241 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [35]:
missing_values = missing_values_table(test_data)
missing_values

Your selected dataframe has 241 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [36]:
train_data.drop(['PayDelay_stdev', 'CharlsonIndexI_stdev'], axis=1, inplace=True)
test_data.drop(['PayDelay_stdev', 'CharlsonIndexI_stdev'], axis=1, inplace=True)

In [37]:
train_data.to_csv("Data/train_data_hhp.csv", encoding='utf-8')
test_data.to_csv("Data/test_data_hhp.csv", encoding='utf-8')

In [2]:
train_data = pd.read_csv('Data/train_data_hhp.csv')
train_data.head()

,MemberID,Year,ClaimsTruncated,DaysInHospital,age_05,age_15,age_25,age_35,age_45,age_55,...,DSFSDrugCountMean,DrugCountMean,DSFSDrugCountRange,DrugCountRange,labCount_min,labCount_max,labCount_ave,labcount_months,labNull,drugNull
0,210,Y1,0,0,False,False,False,True,False,False,...,3.333333,1.666667,4.0,1.0,2.0,2.0,2.0,1.0,False,False
1,3197,Y1,0,0,True,False,False,False,False,False,...,7.750000,1.250000,10.0,1.0,0.0,0.0,0.0,0.0,False,False
2,3889,Y1,0,0,False,False,False,False,False,False,...,6.000000,3.000000,10.0,3.0,10.0,10.0,10.0,1.0,False,False
3,4187,Y1,0,0,False,False,False,False,False,True,...,5.500000,6.100000,9.0,4.0,0.0,0.0,0.0,0.0,False,False
4,9063,Y1,0,0,False,False,False,False,False,False,...,1.500000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,False,False


In [3]:
train_data.columns.tolist()

['MemberID',
 'Year',
 'ClaimsTruncated',
 'DaysInHospital',
 'age_05',
 'age_15',
 'age_25',
 'age_35',
 'age_45',
 'age_55',
 'age_65',
 'age_75',
 'age_85',
 'age_MISS',
 'sexMALE',
 'sexFEMALE',
 'sexMISS',
 'no_Claims',
 'no_Providers',
 'no_Vendors',
 'no_PCPs',
 'no_PlaceSvcs',
 'no_Specialities',
 'no_PrimaryConditionGroups',
 'no_ProcedureGroups',
 'PayDelay_max',
 'LOS_max',
 'dsfs_max',
 'CharlsonIndexI_max',
 'PayDelay_min',
 'LOS_min',
 'dsfs_min',
 'CharlsonIndexI_min',
 'PayDelay_ave',
 'LOS_ave',
 'dsfs_ave',
 'CharlsonIndexI_ave',
 'LOS_stdev',
 'dsfs_stdev',
 'dsfs_range',
 'CharlsonIndexI_range',
 'los_tot_unknown',
 'los_tot_suppressed',
 'los_tot_known',
 'pcg0',
 'pcg1',
 'pcg2',
 'pcg3',
 'pcg4',
 'pcg5',
 'pcg6',
 'pcg7',
 'pcg8',
 'pcg9',
 'pcg10',
 'pcg11',
 'pcg12',
 'pcg13',
 'pcg14',
 'pcg15',
 'pcg16',
 'pcg17',
 'pcg18',
 'pcg19',
 'pcg20',
 'pcg21',
 'pcg22',
 'pcg23',
 'pcg24',
 'pcg25',
 'pcg26',
 'pcg27',
 'pcg28',
 'pcg29',
 'pcg30',
 'pcg31',
 'pcg3

In [39]:
test_data = pd.read_csv('Data/test_data_hhp.csv')
test_data.head() 

,MemberID,Year,ClaimsTruncated,DaysInHospital,age_05,age_15,age_25,age_35,age_45,age_55,...,DSFSDrugCountMean,DrugCountMean,DSFSDrugCountRange,DrugCountRange,labCount_min,labCount_max,labCount_ave,labcount_months,labNull,drugNull
0,4,Y2,0,0,True,False,False,False,False,False,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,False,False
1,210,Y2,0,0,False,False,False,True,False,False,...,0.0,0.00,0.0,0.0,1.0,1.0,1.0,1.0,False,False
2,3197,Y2,0,0,True,False,False,False,False,False,...,1.5,1.50,1.0,1.0,2.0,2.0,2.0,1.0,False,False
3,3457,Y2,0,0,True,False,False,False,False,False,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,False,False
4,3713,Y2,0,0,False,False,False,False,True,False,...,2.5,4.25,3.0,5.0,1.0,8.0,4.5,2.0,False,False
